###**Model calculation: Molecular interferometer**
  
> This code can be used to reproduce the data plotted in Figure 21 of Schultz et al., Coherence in chemistry: foundations and frontiers. *Chem. Revs.* (submitted)

To begin, press the play button beside each cell (proceed sequentially through the cells).

In [ ]:
#@title Import libraries, prep plot settings

import matplotlib.pyplot as plt
import numpy as np
from math import *
import cmath
from scipy import linalg
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

hbar = 1

i_ = complex(0, 1)

SMALL_SIZE = 18
MEDIUM_SIZE = 20
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [95]:
#@title Tunable parameters

V = 0.5
V_phase_L = 0
V_phase_R = np.arange(0,4*pi,0.1,dtype=float)
site_energies = [0, 2, 2, -2]
points = 1000

In [96]:
#@title Set up arrays and functions
Delta_phi = V_phase_R-V_phase_L

DB1coupling = V*np.exp(-i_*V_phase_L)
B1Acoupling = V*np.exp(-i_*V_phase_L)
B2Acoupling = V*np.exp(-i_*V_phase_L)

rho_init = np.array([[1, 0, 0, 0], [0, 0, 0, 0], [ 0, 0, 0, 0], [0, 0, 0, 0]], dtype='complex') # Initial density matrix
sys_size = rho_init.shape[0]

def dag(A):
  return np.conjugate(A)

def observable(P,B):
  return np.real(np.trace(B @ P))    # expectation value is the trace of the projection (P) onto B (rho eig)

proj = np.zeros([sys_size,sys_size])  # Projection onto site 4 (or the population on the acceptor) is the observable
proj[3,3] = 1

In [97]:
#@title Calculate unitary time-evolution of density matrix (in the site basis)

maxpop = []
for n in range(len(V_phase_R)):

  DB2coupling = V*np.exp(-i_*V_phase_R[n])

  H = np.array([[site_energies[0], DB1coupling, DB2coupling, 0], [dag(DB1coupling), site_energies[1], 0, B1Acoupling], [dag(DB2coupling), 0, site_energies[2], B2Acoupling], [0, dag(B1Acoupling), dag(B2Acoupling), site_energies[3]]], dtype='complex')   # Hamiltonian

  maxEdiff = np.max(np.abs(np.diff(H)))
  tmax = 25/maxEdiff
  t = np.linspace(0,tmax,points)    # Time vector
  dt = t[1]-t[0]                    # Time step

  U = linalg.expm(-i_*H*dt)         # Propagators
  Udag = linalg.expm(i_*H*dt)

  rho_store = []
  rho_store.append(rho_init)
  population = []
  population.append(observable(proj,rho_store[0]))
  rho_time = rho_init.copy()

  for i in range(len(t)-1):
    k = i+1
    rho_time = U @ rho_time @ Udag                      # Propagate the density matrix by dt
    rho_store.append(rho_time)                          # Store propagated density matrix
    population.append(observable(proj,rho_store[k]))    # Calculate the observable

  maxpop.append(np.max(population))

In [ ]:
#@title Plot

fig = plt.figure(figsize=(15,6))
ax1 = fig.add_subplot(1, 2, 1)
ax1.plot(t,population, 'r-', linewidth=1)
plt.title("Example population dynamics")
plt.xlabel("time (arb. units)")
plt.ylabel("Acceptor population")

ax2 = fig.add_subplot(1, 2, 2)
ax2.plot(Delta_phi,maxpop, 'r-', linewidth=1)
ax2.xaxis.set_ticks([0,pi,2*pi,3*pi, 4*pi])
plt.title("Coupling pathway interference")
plt.xlabel("Phase difference ($\Delta \phi_{LR}$)")
plt.ylabel("Acceptor population")

fig.tight_layout()